In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
# import altair as alt
import math

In [2]:
recruitment_compensation = pd.read_csv("../data_original/24_Recruitment_compensation_anon.csv")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# isolating org column and deleting rows before the compensation data
org_column = recruitment_compensation["Organization"]

# compensation and pay equity begins in column 45
df = recruitment_compensation.iloc[: , 45:]
df.columns

Index(['A publicly-stated philosophy and commitment to pay transparency', 'Transparency of pay process, including salary bands/ranges that are shared with all employees', 'Compensation reviews are performed separately from performance reviews', 'The organization performs a "job leveling" or compensation analysis at least every two years', 'None.1', 'Other.2', 'Compensation & Pay Equity: Do you have an organization-wide written policy that prohibits asking or otherwise collecting prior compensation data from job candidates?', 'Compensation and Pay Equity: Has your organization conducted a pay equity assessment (comparable compensation for same work) in the last 12 months? ', 'Gender identity', 'Race/Ethnicity', 'Economic status', 'Religion', 'Sexual orientation', 'Ability', 'None.2', 'Salary', 'Bonuses', 'Equity or other Ownership shares', 'Company perks (company cars, professional membership fees, etc)', 'Reimbursement', 'Other.3',
       'Compensation & Pay Equity: What were the overa

In [3]:
df

,A publicly-stated philosophy and commitment to pay transparency,"Transparency of pay process, including salary bands/ranges that are shared with all employees",Compensation reviews are performed separately from performance reviews,"The organization performs a ""job leveling"" or compensation analysis at least every two years",None.1,Other.2,Compensation & Pay Equity: Do you have an organization-wide written policy that prohibits asking or otherwise collecting prior compensation data from job candidates?,Compensation and Pay Equity: Has your organization conducted a pay equity assessment (comparable compensation for same work) in the last 12 months?,Gender identity,Race/Ethnicity,Economic status,Religion,Sexual orientation,Ability,None.2,Salary,Bonuses,Equity or other Ownership shares,"Company perks (company cars, professional membership fees, etc)",Reimbursement,Other.3,Compensation & Pay Equity: What were the overall results?,Entry Level,Management/ Supervisory Level,Executive Management,C-suite Level,None.3,Unsure,"Compensation & Pay Equity: If your board is compensated, have you evaluated pay equity at the board level?",Black or African American women,Latina women,Asian women,"Other (Native American/Pacific Islander, Alaskan)",Mixed Race,White/Caucasian women,Working moms (of school-aged children),LGTBQ women,Gender non-binary,Unsure.1,None.4,No.1,With C-Suite,With Board/Governing Body,With Managers,With All Workers,Publicly,No.2,With C-Suite.1,With Board/Governing Body.1,With Managers.1,With All Workers.1,"Compensation & Pay Equity: Has your organization created a plan to address pay-equity issues with goals, timeline, and activities to ensure success?","Compensation & Pay Equity: If yes, have you achieved your goals?",% Total Workers who received compensation (excluding COLA) adjustment in the last 12 months:,% Workers who identify as Women,Women who identify as Black or African American.1,Women who identify as Latina.1,Women who identify as Asian.1,Women who identify as Native American/Alaskan/Pacific Islander.1,Women who identify as Mixed Race.1,Women who identify as White/Caucasian.1,Women who identify as Disabled.1,Women who identify as LGBTQ.1,Persons who identify as gender non-binary.1,None.5,Give us feedback - how challenging is it to provide demographic details for your workforce to the degree asked above?,"Compensation and Pay Equity: If your organization has a stock-option program or employee ownership program, have you evaluated whether shares have been allocated equitably across genders?","Compensation & Pay Equity: Is the lowest paid woman - part-time, full-time or on contract - paid a living wage?","Compensation & Pay Equity: Before we move to the next section, is there anything else you want to tell us about how your organization tracks, reports on or is working to improve its compensation and pay equity structures?",Organization,benchmark_group,number_of_employees,number_of_years,workforce
0,NaN,NaN,Compensation reviews are performed separately ...,"The organization performs a ""job leveling"" or ...",NaN,NaN,Yes,In development,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Progress,N/A - No targets were established,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unsure/ responses not provided for compensatio...,No evaluation conducted,Yes,NaN,Company 1X,Technology,"1000-4,999 Employees",5-14 years,At least 80% of employees are Salaried
1,NaN,NaN,NaN,NaN,None,NaN,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prior questions should just be N/A as everyone...,N/A (no plan),Yes,NaN,Company 1W,NonProfit,Fewer than 100 Employees,15 or more years,We have a mixed workforce of hourly and salari...
2,A pu

In [4]:
#Renaming columns based on Question # followed by _A/B/C/etc if a multiple selection
#Question numbering is from the survey at: https://gender-ideal.org/the-assessment

df.rename(columns={'A publicly-stated philosophy and commitment to pay transparency':'Q21_A',
                   'Transparency of pay process, including salary bands/ranges that are shared with all employees':'Q21_B',
                   'Compensation reviews are performed separately from performance reviews':'Q21_C',
                   'The organization performs a "job leveling" or compensation analysis at least every two years':'Q21_D',
                   'None.1':'Q21_None',
                   'Other.2':'Q21_Other',
                   'Compensation & Pay Equity: Do you have an organization-wide written policy that prohibits asking or otherwise collecting prior compensation data from job candidates?':'Q22',
                   'Compensation and Pay Equity: Has your organization conducted a pay equity assessment (comparable compensation for same work) in the last 12 months? ':'Q23', 
                   'Gender identity':'Q24_A',
                   'Race/Ethnicity':'Q24_B',
                   'Economic status':'Q24_C',
                   'Religion':'Q24_D',
                   'Sexual orientation':'Q24_E',
                   'Ability':'Q24_F',
                   'None.2':'Q24_None',
                   'Salary':'Q25_A',
                   'Bonuses':'Q25_B',
                   'Equity or other Ownership shares':'Q25_C',
                   'Company perks (company cars, professional membership fees, etc)':'Q25_D',
                   'Reimbursement':'Q25_E',
                   'Other.3':'Q25_Other',
                   'Compensation & Pay Equity: What were the overall results?':'Q26',
                   'Entry Level':'Q27_A',
                   'Management/ Supervisory Level':'Q27_B',
                   'Executive Management':'Q27_C',
                   'C-suite Level':'Q27_D',
                   'None.3':'Q27_None',
                   'Unsure':'Q27_E',
                   'Compensation & Pay Equity: If your board is compensated, have you evaluated pay equity at the board level?':'Q28',
                   'Black or African American women':'Q29_A',
                   'Latina  women':'Q29_B',
                   'Asian women':'Q29_C',
                   'Other (Native American/Pacific Islander, Alaskan)':'Q29_D',
                   'Mixed Race':'Q29_E',
                   'White/Caucasian women':'Q29_F',
                   'Working moms (of school-aged children)':'Q29_G',
                   'LGTBQ women':'Q29_H',
                   'Gender non-binary':'Q29_I',
                   'Unsure.1':'Q29_J',
                   'None.4':'Q29_None',
                   'No.1':'Q30_A',
                   'With C-Suite':'Q30_B',
                   'With Board/Governing Body':'Q30_C',
                   'With Managers':'Q30_D',
                   'With All Workers':'Q30_E',
                   'Publicly':'Q30_F',
                   'No.2':'Q31_A',
                   'With C-Suite.1':'Q31_B',
                   'With Board/Governing Body.1':'Q31_C',
                   'With Managers.1':'Q31_D',
                   'With All Workers.1':'Q31_E',
                   'Compensation & Pay Equity: Has your organization created a plan to address pay-equity issues with goals, timeline, and activities to ensure success?':'Q32',
                   'Compensation & Pay Equity: If yes, have you achieved your goals?':'Q33',
                   '% Total Workers who received compensation (excluding COLA) adjustment in  the last 12 months:':'Q34_A',
                   '% Workers who identify as Women':'Q34_B',
                   'Women who identify as Black or African American.1':'Q34_C_A',
                   'Women who identify as Latina.1':'Q34_C_B',
                   'Women who identify as Asian.1':'Q34_C_C',
                   'Women who identify as Native American/Alaskan/Pacific Islander.1':'Q34_C_D',
                   'Women who identify as Mixed Race.1':'Q34_C_E',
                   'Women who identify as White/Caucasian.1':'Q34_C_F',
                   'Women who identify as Disabled.1':'Q34_C_G',
                   'Women who identify as LGBTQ.1':'Q34_C_H',
                   'Persons who identify as gender non-binary.1':'Q34_C_I',
                   'None.5':'Q34_C_None',
                   'Give us feedback - how challenging is it to provide demographic details for your workforce to the degree asked above?':'Q34_D',
                   'Compensation and Pay Equity: If your organization has a stock-option program or employee ownership program, have you evaluated whether shares have been allocated equitably across genders? ':'Q35',
                   'Compensation & Pay Equity: Is the lowest paid woman - part-time, full-time or on contract - paid a living wage?':'Q36',
                   'Compensation & Pay Equity: Before we move to the next section, is there anything else you want to tell us about how your organization tracks, reports on or is working to improve its compensation and pay equity structures?':'Q37',},inplace=True)

df

,Q21_A,Q21_B,Q21_C,Q21_D,Q21_None,Q21_Other,Q22,Q23,Q24_A,Q24_B,Q24_C,Q24_D,Q24_E,Q24_F,Q24_None,Q25_A,Q25_B,Q25_C,Q25_D,Q25_E,Q25_Other,Q26,Q27_A,Q27_B,Q27_C,Q27_D,Q27_None,Q27_E,Q28,Q29_A,Q29_B,Q29_C,Q29_D,Q29_E,Q29_F,Q29_G,Q29_H,Q29_I,Q29_J,Q29_None,Q30_A,Q30_B,Q30_C,Q30_D,Q30_E,Q30_F,Q31_A,Q31_B,Q31_C,Q31_D,Q31_E,Q32,Q33,Q34_A,Q34_B,Q34_C_A,Q34_C_B,Q34_C_C,Q34_C_D,Q34_C_E,Q34_C_F,Q34_C_G,Q34_C_H,Q34_C_I,Q34_C_None,Q34_D,Q35,Q36,Q37,Organization,benchmark_group,number_of_employees,number_of_years,workforce
0,NaN,NaN,Compensation reviews are performed separately ...,"The organization performs a ""job leveling"" or ...",NaN,NaN,Yes,In development,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Progress,N/A - No targets were established,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unsure/ responses not provided for compensatio...,No evaluation conducted,Yes,NaN,Company 1X,Technology,"1000-4,999 Employees",5-14 years,At least 80% of employees are Salaried
1,NaN,NaN,NaN,NaN,None,NaN,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prior questions should just be N/A as everyone...,N/A (no plan),Yes,NaN,Company 1W,NonProfit,Fewer than 100 Employees,15 or more years,We have a mixed workforce of hourly and salari...
2,A publicly-stated philosophy and commitment to...,NaN,NaN,NaN,NaN,NaN,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Compensation adjustments happen in September s...,No evaluation conducted,Yes,NaN,Company 1V,Finance,Fewer than 100 Employees,5-14 years,At least 80% of employees are Salaried
3,NaN,NaN,Compensation reviews are performed separately ...,"The organization performs a ""job leveling"" or ...",NaN,NaN,Yes,In development,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,57.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Company 1U,Finance,Fewer than 100 Employees,15 or more years,At least 80% of employees are Salaried
4,NaN,NaN,Compensation reviews are performed separately ...,"The organization performs a ""job leveling"" or ...",NaN,NaN,No,Yes,Gender identity,Race/Ethnicity,NaN,NaN,NaN,NaN,NaN,Salary,Bonuses,Equity or other Ownership shares,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A (no board compensation),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,With C-Suite,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,"Yes, shared internally with employees",Goals met,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No evaluation conducted,Yes,NaN,Company 1T,Technology,100-249 Employees,5-14 years,We have a mixed workforce of hourly and salari...
5,NaN,NaN,Compensation reviews are performed separately ...,"The organization performs a ""job leveling"" or ...",NaN,NaN,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,54.0,69.0,NaN,NaN,Women who identify as Asian,NaN,Women who identify as Mixed Race,NaN,NaN,NaN,NaN,NaN,NaN,N/A (no plan),Yes,NaN,Company 1S,Technology,"1000-4,999 Employees",Fewer than 5 years,We have a mixed workforce of hourly and salari...
6,A publicly-stated philosophy and commitment to...,NaN,Compensation reviews are performed separately ...,"The organization performs a ""job leveling"" or ...",NaN,NaN,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [5]:
df.columns

Index(['Q21_A', 'Q21_B', 'Q21_C', 'Q21_D', 'Q21_None', 'Q21_Other', 'Q22', 'Q23', 'Q24_A', 'Q24_B', 'Q24_C', 'Q24_D', 'Q24_E', 'Q24_F', 'Q24_None', 'Q25_A', 'Q25_B', 'Q25_C', 'Q25_D', 'Q25_E', 'Q25_Other', 'Q26', 'Q27_A', 'Q27_B', 'Q27_C', 'Q27_D', 'Q27_None', 'Q27_E', 'Q28', 'Q29_A', 'Q29_B', 'Q29_C', 'Q29_D', 'Q29_E', 'Q29_F', 'Q29_G', 'Q29_H', 'Q29_I', 'Q29_J', 'Q29_None', 'Q30_A', 'Q30_B', 'Q30_C', 'Q30_D', 'Q30_E', 'Q30_F', 'Q31_A', 'Q31_B', 'Q31_C', 'Q31_D', 'Q31_E', 'Q32', 'Q33', 'Q34_A', 'Q34_B', 'Q34_C_A', 'Q34_C_B', 'Q34_C_C', 'Q34_C_D', 'Q34_C_E', 'Q34_C_F', 'Q34_C_G', 'Q34_C_H', 'Q34_C_I', 'Q34_C_None', 'Q34_D', 'Q35', 'Q36', 'Q37', 'Organization', 'benchmark_group', 'number_of_employees', 'number_of_years', 'workforce'], dtype='object')

In [6]:
np.isnan(df['Q24_A'][0])

True

In [7]:
# # if 23 "No" or "In development", make 24-31 "N/A - No pay equity assessment conducted"
# for row in range(0,len(df)):
#     if df['Q23'][row] == 'No' or df['Q23'][row] == "In development":
#         for col in range(8, 51):
#             temp = df[col].replace(np.nan, 'N/A - No pay equity assessment conducted', regex=True)
#             df[col] = temp

for col in range(8, 51):
    question = df.columns[col]
    temp = df[question].replace(np.nan, 'N/A - No pay equity assessment conducted', regex=True)
    df[question] = temp 
            
df

,Q21_A,Q21_B,Q21_C,Q21_D,Q21_None,Q21_Other,Q22,Q23,Q24_A,Q24_B,Q24_C,Q24_D,Q24_E,Q24_F,Q24_None,Q25_A,Q25_B,Q25_C,Q25_D,Q25_E,Q25_Other,Q26,Q27_A,Q27_B,Q27_C,Q27_D,Q27_None,Q27_E,Q28,Q29_A,Q29_B,Q29_C,Q29_D,Q29_E,Q29_F,Q29_G,Q29_H,Q29_I,Q29_J,Q29_None,Q30_A,Q30_B,Q30_C,Q30_D,Q30_E,Q30_F,Q31_A,Q31_B,Q31_C,Q31_D,Q31_E,Q32,Q33,Q34_A,Q34_B,Q34_C_A,Q34_C_B,Q34_C_C,Q34_C_D,Q34_C_E,Q34_C_F,Q34_C_G,Q34_C_H,Q34_C_I,Q34_C_None,Q34_D,Q35,Q36,Q37,Organization,benchmark_group,number_of_employees,number_of_years,workforce
0,NaN,NaN,Compensation reviews are performed separately ...,"The organization performs a ""job leveling"" or ...",NaN,NaN,Yes,In development,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,In Progress,N/A - No targets were established,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unsure/ responses not provided for compensatio...,No evaluation conducted,Yes,NaN,Company 1X,Technology,"1000-4,999 Employees",5-14 years,At least 80% of employees are Salaried
1,NaN,NaN,NaN,NaN,None,NaN,No,No,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - 

In [8]:
# 33 is 'N/A - No targets were established' if 32 was 'No'
temp = df['Q33'].replace(np.nan, 'N/A - No targets were established', regex=True)
df['Q33'] = temp 
df['Q33']

0      N/A - No targets were established
1      N/A - No targets were established
2      N/A - No targets were established
3      N/A - No targets were established
4                              Goals met
5      N/A - No targets were established
6      N/A - No targets were established
7      N/A - No targets were established
8      N/A - No targets were established
9                              Goals met
10     N/A - No targets were established
11     N/A - No targets were established
12     N/A - No targets were established
13     N/A - No targets were established
14     N/A - No targets were established
15    Nearing goals (66-99% targets met)
16     N/A - No targets were established
17     N/A - No targets were established
18     N/A - No targets were established
19     N/A - No targets were established
20     N/A - No targets were established
21     N/A - No targets were established
22     N/A - No targets were established
23     N/A - No targets were established
Name: Q33, dtype

In [9]:
df

,Q21_A,Q21_B,Q21_C,Q21_D,Q21_None,Q21_Other,Q22,Q23,Q24_A,Q24_B,Q24_C,Q24_D,Q24_E,Q24_F,Q24_None,Q25_A,Q25_B,Q25_C,Q25_D,Q25_E,Q25_Other,Q26,Q27_A,Q27_B,Q27_C,Q27_D,Q27_None,Q27_E,Q28,Q29_A,Q29_B,Q29_C,Q29_D,Q29_E,Q29_F,Q29_G,Q29_H,Q29_I,Q29_J,Q29_None,Q30_A,Q30_B,Q30_C,Q30_D,Q30_E,Q30_F,Q31_A,Q31_B,Q31_C,Q31_D,Q31_E,Q32,Q33,Q34_A,Q34_B,Q34_C_A,Q34_C_B,Q34_C_C,Q34_C_D,Q34_C_E,Q34_C_F,Q34_C_G,Q34_C_H,Q34_C_I,Q34_C_None,Q34_D,Q35,Q36,Q37,Organization,benchmark_group,number_of_employees,number_of_years,workforce
0,NaN,NaN,Compensation reviews are performed separately ...,"The organization performs a ""job leveling"" or ...",NaN,NaN,Yes,In development,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,In Progress,N/A - No targets were established,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unsure/ responses not provided for compensatio...,No evaluation conducted,Yes,NaN,Company 1X,Technology,"1000-4,999 Employees",5-14 years,At least 80% of employees are Salaried
1,NaN,NaN,NaN,NaN,None,NaN,No,No,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - No pay equity assessment conducted,N/A - 

In [10]:
#Write out to csv
df.to_csv("C:/Users/Admin/Desktop/gender-ideal-main/clean_final/compensation_clean_final.csv")